### LISA glitches pipeline v6

#### Summary

This Jupyter notebook runs a pipeline which injects glitch(es), GWB or NEA, runs a simulation and calculates the TDI combinations

#### Inputs

* Glitch simualtion variables (standard types from LISA Glitch supported)
* GWB simualtion variables
* NEA simualtion variables

#### Outputs

* Glitch file
* Measurements file
* Michelson and orthogonal (1st and 2nd order) TDI combinations
* Combined plots of glitch(es) and TDI combinations

#### Processing

1. Setup environment
1. Declare functions
1. Setup logging and simulation inputs
1. Define simulation parameters
1. Create GW and calculate response
1. Create glitch(es) for injection
1. Run simulation
1. Calculate TDI combinations
1. Plot glitch(es), GW responses and TDI combinations

#### Constraints

* None

#### Acknowledgements

* LISA Orbits
* LISA Glitch
* LISA Instrument
* PyTDI

#### References

[1] None

### Setup environment

#### Install useful Python packages


Create a new virtual environment, activate it, upgrade pip and install the required packages
```shell
python3 -m venv venv
source venv/bin/activate
python -m pip install --upgrade pip
pip install numpy scipy h5py matplotlib jupyter ipynb ipykernel pandas tqdm
```

In [ ]:
from datetime import datetime
from h5py import File
from importlib import reload
import logging
import matplotlib.pyplot as plt
import numpy as np
from numpy import sin, cos, pi, sqrt, dot
from numpy.random import random
import os
import pandas as pd
from scipy.spatial.transform import Rotation as R
from threading import Thread
from tqdm import tqdm

# from ipynb.fs.defs import common
# from ipynb.fs.defs import utils
import common
import utils

#### Install LISA Simulation packages

Install packages from LISA Consortium's repo (git credentials required) using the simulation workshop versions (March 2022)

```shell
pip install git+https://gitlab.in2p3.fr/lisa-simulation/constants.git@v1.2
pip install git+https://gitlab.in2p3.fr/lisa-simulation/glitch.git@v1.0
pip install git+https://gitlab.in2p3.fr/lisa-simulation/gw-response.git@v1.0.1
pip install git+https://gitlab.in2p3.fr/lisa-simulation/instrument.git@v1.0.4
pip install pytdi
```

or using my not-preferred versions (abandoned in May 2022 in favour of workshop versions)

```shell
pip install git+https://gitlab.in2p3.fr/lisa-simulation/constants.git@v1.2
pip install git+https://gitlab.in2p3.fr/lisa-simulation/glitch.git@v1.1
pip install git+https://gitlab.in2p3.fr/lisa-simulation/gw-response.git@v1.1
pip install git+https://gitlab.in2p3.fr/lisa-simulation/instrument.git@v1.0.7
pip install pytdi
```

or using latest versions

```shell
pip install git+https://gitlab.in2p3.fr/lisa-simulation/constants.git@latest
pip install git+https://gitlab.in2p3.fr/lisa-simulation/glitch.git@latest
pip install git+https://gitlab.in2p3.fr/lisa-simulation/gw-response.git@latest
pip install git+https://gitlab.in2p3.fr/lisa-simulation/instrument.git@latest
pip install pytdi
```

In [ ]:
import lisaconstants
import lisainstrument
import lisagwresponse
import lisaglitch 
import pytdi

from lisaconstants import ASTRONOMICAL_UNIT, GRAVITATIONAL_CONSTANT, SPEED_OF_LIGHT, GM_SUN
from lisainstrument import Instrument
from lisagwresponse import ReadStrain
from lisaglitch import StepGlitch, RectangleGlitch, ShapeletGlitch, IntegratedShapeletGlitch
from lisaglitch import TimeSeriesGlitch
from lisaglitch import OneSidedDoubleExpGlitch, TwoSidedDoubleExpGlitch 
if lisaglitch.__version__=='1.1': from lisaglitch import IntegratedOneSidedDoubleExpGlitch, IntegratedTwoSidedDoubleExpGlitch
from pytdi import Data, michelson, ortho

### Setup

In [ ]:
# Reload my local libraries to save restarting the kernel whenever a library changes
common = reload(common)
utils = reload(utils)

common.check_versions()

logging.basicConfig()

### Set variables common to all simulations with glitch(es), NEAs and GW responses

In [ ]:
# Parameters cell for Papermill

dataset_name = 'df1'
timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
first_instance = 0
instances_to_do = 3000

In [ ]:
LINKS = ['12', '23', '31', '13', '32', '21']

root_dir = '/Users/stevemead/LISAandNEA'
orbits_path = f'{root_dir}/is-equalarmlength-orbits.h5'

run_id = f'{dataset_name}_{timestamp}'
last_instance = first_instance+instances_to_do

output_dir = f'{root_dir}/datasets/{run_id}'

In [ ]:
def get_output_file_name(output_file, file_type):
    return utils.get_output_file_name(output_dir, f'{output_file}_{run_id}', file_type)

In [ ]:
utils.create_directory(output_dir)

In [ ]:

orbits_t0 = utils.get_attr(orbits_path, 't0')
orbits_tsize = utils.get_attr(orbits_path, 'tsize')
orbits_dt = utils.get_attr(orbits_path, 'dt')
orbits_tduration = utils.get_attr(orbits_path, 'tduration')

# esa-orbits.h5 (v1.0.2)
#   dt        =       8,640 s   = 0.1 days  (2.4 hours)
#   tduration = 123,033,600 s   = 4.0 years (ish)

print(f'Orbits file ({orbits_path}):\n t0: {orbits_t0} s\n size: {orbits_tsize} samples\n sampling interval: {orbits_dt} s\n duration: {orbits_tduration} s')
# utils.describe_hdf5(orbits_path)

In [ ]:
# set common parameters for the simulation with injection of all glitch(es) and GW responses
duration = 1500 # seconds
fs = 16 # Hz

### Create the dataframe with the variables used to create the dataset

In [ ]:
# a pandas dataframe is used to store the variables used to create each instance in the dataset
df_path = f'{root_dir}/datasets/{dataset_name}.txt'
df = pd.read_csv(df_path)
total_instances = df.shape[0]

In [ ]:
# Create the numpy array used to store the output dataset
# parameterise the 600

# dataset = np.zeros((total_instances,600,6))

In [ ]:
def plot_me(series, title=''):
    fig = plt.subplot()
    plt.plot(series)
    fig.set(title=title)
    plt.show()

In [ ]:
def process_instance(index):
    
    instance = df.iloc[index]
    t_shift = 800
    t_inj = orbits_t0 + t_shift + instance.t_random # seconds (makes a round number w.r.t. t0 in the esa-orbits file of t = 13,000 s +/- 10 s)

    if instance.type == 'glitch':
        
        logging.getLogger('lisaglitch').setLevel(logging.WARNING)
        
        # make standard glitch(es)
        # Bayle (2019, sect 7.2.2 bullet 2) amplitude = 10 pm s-1, pm = 1e-12 m so approx. 1e-11
        g1 = common.make_glitch(instance.glitch_type, duration=duration, t_inj=t_inj, t0=orbits_t0, fs=fs, inj_point=instance.glitch_inj_point, level=instance.glitch_level, width=instance.glitch_width, beta=instance.glitch_beta) 
        glitches = [g1]

        # for g in glitches: utils.describe_glitch(g)
        glitch_path = common.save_glitch_file(glitches, get_output_file_name(f'glitch_{index}', 'h5'))
        # utils.describe_hdf5(glitch_path)
    
    elif instance.type == 'nea':
        
        logging.getLogger('lisaglitch').setLevel(logging.WARNING)

        if instance.nea_model == 'pair_of_glitches_1':

            inj_point1, inj_point2 = common.get_inj_points(instance.nea_sc)
            g1 = common.make_glitch(instance.glitch_type, duration=duration, t_inj=t_inj, t0=orbits_t0, fs=fs, inj_point=inj_point1, level=instance.glitch_level, width=instance.glitch_width) 
            g2 = common.make_glitch(instance.glitch_type, duration=duration, t_inj=t_inj, t0=orbits_t0, fs=fs, inj_point=inj_point2, level=instance.glitch_level*instance.nea_glitch_ratio, width=instance.glitch_width)
            glitches = [g1,g2]
        
        elif instance.nea_model == 'pair_of_glitches_2':

            inj_point1, inj_point2 = common.get_inj_points(instance.nea_sc)
            g1 = common.make_glitch(instance.glitch_type, duration=duration, t_inj=t_inj, t0=orbits_t0, fs=fs, inj_point=inj_point1, level=instance.glitch_level*instance.nea_glitch_ratio, width=instance.glitch_width) 
            g2 = common.make_glitch(instance.glitch_type, duration=duration, t_inj=t_inj, t0=orbits_t0, fs=fs, inj_point=inj_point2, level=instance.glitch_level, width=instance.glitch_width)
            glitches = [g1,g2]

        elif instance.nea_model == 'nea':
            
            nea = dict(
                M=instance.nea_M, 
                D=instance.nea_D, 
                V=instance.nea_V, 
                angles=[
                    instance.nea_angle_X,
                    instance.nea_angle_Y,
                    instance.nea_angle_Z
                ],
            )
            glitches = common.make_neaglitches(nea=nea, t_inj=t_inj, t0=orbits_t0, duration=duration, sc=instance.nea_sc)
            
        else:
            ValueError('Unknown NEA model')
        
        # for g in glitches: utils.describe_glitch(g)
        glitch_path = common.save_glitch_file(glitches, get_output_file_name(f'glitch_{index}', 'h5'))
        # utils.describe_hdf5(glitch_path)

    elif instance.type == 'gwburst':

        # Waveforms are computed by LISA GW Response at the Sun, and then “transformed” to the emitter and receiver spacecraft
        # This transformation changes, depending on the GW source localization in the sky and the spacecrafts' positions 

        t_transformation = common.estimate_transformation(orbits_path, gw_beta=instance.gw_beta, gw_lambda=instance.gw_lambda)

        print(f'Analysis window for the GW response will be transformed by approximately[1]: {t_transformation:.1f} s')
        print(f' [1] because the GW is injected at t_inj, not at t=0')

        t, hplus, hcross = common.make_gwburst(width=instance.gw_width, level=instance.gw_level, t_inj=t_inj-t_transformation, orbits_t0=orbits_t0, duration=duration, fs=fs, model=instance.gw_model, sigma=instance.gw_sigma)

        source = ReadStrain(
            t, hplus, hcross,
            orbits=orbits_path,
            gw_beta=instance.gw_beta, gw_lambda=instance.gw_lambda,
            size=duration*fs, dt=1/fs, t0=orbits_t0 + 10,
        )

        # source.plot()
        gw_responses = source.compute_gw_response(links=LINKS,t=source.t)
        gws_path = common.save_gws_file(source, get_output_file_name(f'gws_{index}', 'h5'))
        # utils.describe_hdf5(gws_path)

    else:
        ValueError('Unknown instance type')

    logging.getLogger('lisainstrument').setLevel(logging.WARNING)
    skip = 1000

    if instance.type in ['glitch', 'nea']:
        instru = common.make_instrument(
                    orbits=orbits_path, 
                    t0=orbits_t0+10, # cannot start at orbits_t0 because the position of the emitting spacecraft must be known at the start of the simulation
                    duration=duration, 
                    glitches=glitch_path)
    else:
        instru = common.make_instrument(
                    orbits=orbits_path, 
                    t0=orbits_t0+10, # cannot start at orbits_t0 because the position of the emitting spacecraft must be known at the start of the simulation
                    duration=duration,
                    gws=gws_path)

    if instance.noise=='none':
        instru.disable_all_noises() 
    elif instance.noise=='laser':
        instru.disable_all_noises(but='laser')
    elif instance.noise=='clock':
        instru.disable_all_noises(but='clock')
    else:
        ValueError('Unknown noise type')

    instru.simulate()

    # utils.describe_instrument(instru, skip=skip)
    measurements_path = common.save_measurements_file(instru, get_output_file_name(f'measurements_{index}', 'h5'))
    # utils.describe_hdf5(measurements_path)

    # write out time-series of the glitch(es) or strain for posterity
    if instance.type in ['glitch','nea']:
        if len(glitches) == 2:
            glitch0_raw = glitches[0].compute_signal(glitches[0].t)
            glitch1_raw = glitches[1].compute_signal(glitches[1].t)
            glitch_raw = np.stack((glitches[0].t, glitch0_raw, glitch1_raw), axis=1)
        else:
            glitch_raw = np.stack((glitches[0].t, glitches[0].compute_signal(glitches[0].t)), axis=1)
        
        glitch_raw_path = get_output_file_name(f'raw_glitch_{index}', 'h5') 
        np.save(glitch_raw_path, glitch_raw)

    else:
        strain_raw_path = get_output_file_name(f'raw_strain_{index}', 'h5')
        strain_raw = np.stack((t, hplus, hcross), axis=1)
        np.save(strain_raw_path, strain_raw)

    all_signals = ['total', 'offsets', 'fluctuations']

    for signals in all_signals:

        data = Data.from_instrument(instru, 
                                    signals=signals, 
                                    skipped=skip
                                    )

        data_X1, data_Y1, data_Z1 = common.make_combinations(data, type='michelson', order=1)
        data_A1, data_E1, data_T1 = common.make_combinations(data, type='ortho', order=1)
        data_X2, data_Y2, data_Z2 = common.make_combinations(data, type='michelson', order=2)
        data_A2, data_E2, data_T2 = common.make_combinations(data, type='ortho', order=2)

        # write out the raw TDI combinations for the whole timeseries
        tdi_raw_path = get_output_file_name(f'raw_tdi_{signals}_{index}', 'npy')
        tdi_raw = np.stack((instru.t[skip:], data_X1, data_Y1, data_Z1, data_A1, data_E1, data_T1, data_X2, data_Y2, data_Z2, data_A2, data_E2, data_T2), axis=1)
        np.save(tdi_raw_path, tdi_raw)

        # write out the TDI combinations, just the datasets for analysis
        tdi_data_path = get_output_file_name(f'data_tdi_{signals}_{index}', 'npy')
        awmin, awmax = int((t_shift - 40 -(int(orbits_t0)+10+int(skip/4)))*4),int((t_shift + 120 -(int(orbits_t0)+10+int(skip/4)))*4) # 2160, 2800
        tdi_data = np.stack((data_X2[awmin:awmax], data_Y2[awmin:awmax], data_Z2[awmin:awmax], data_A2[awmin:awmax], data_E2[awmin:awmax], data_T2[awmin:awmax]), axis=1)
        np.save(tdi_data_path, tdi_data)

    # finally, plot some graphs off of the final set of TDI combinations (fluctuations)
    fig, axs = plt.subplots(nrows=6, 
                        ncols=1, 
                        # sharex='col', 
                        sharey='row', 
                        squeeze=False, 
                        figsize=(12, 10)) 
    
    if instance.type in ['glitch','nea']:
        utils.plot_blank(axs, row=0)
        utils.plot_glitches(glitches[0].t, glitches, axs, row=1)
    else:
        utils.plot_strain(t, hplus, hcross, axs, row=0)
        utils.plot_gws(source.t, gw_responses, axs, row=1)

    utils.plot_combos(instru.t[skip:], data_X1, data_Y1, data_Z1, ['$X₁$', '$Y₁$', '$Z₁$'], axs, row=2, ylabel='Michelson 1st')
    utils.plot_combos(instru.t[skip:], data_A1, data_E1, data_T1, ['$A₁$', '$E₁$', '$T₁$'], axs, row=3, ylabel='Orthogonal 1st')
    utils.plot_combos(instru.t[skip:], data_X2, data_Y2, data_Z2, ['$X₂$', '$Y₂$', '$Z₂$'], axs, row=4, ylabel='Michelson 2nd')
    utils.plot_combos(instru.t[skip:], data_A2, data_E2, data_T2, ['$A₂$', '$E₂$', '$T₂$'], axs, row=5, ylabel='Orthogonal 2nd')
    
    utils.format_fig(fig, axs)
    
    # Plot the graphs over an analysis window with common width but different xmin/xmax for the transformed GW burst
    xmin = orbits_t0 + t_shift - 40
    xmax = orbits_t0 + t_shift + 120
    for i in range(axs.shape[0]): 
        for j in range(axs.shape[1]):
            if (instance.type=='gwburst') & (i==0):
                axs[i,j].set(xlim=(xmin-t_transformation, xmax-t_transformation))
            else:
                axs[i,j].set(xlim=(xmin, xmax))
    
    if instance.type == 'gwburst':
        axs[1,0].legend(loc='upper right', ncol=2)

    fig_path = get_output_file_name(f'all_combos_{index}', 'png')
    plt.savefig(fig_path)
    # plt.show()
    plt.close()

    # plot_me(data_X2[awmin:awmax], title='X2')
    # plot_me(data_Y2[awmin:awmax], title='Y2')
    # plot_me(data_Z2[awmin:awmax], title='Z2')
    # plot_me(data_A2[awmin:awmax], title='A2')
    # plot_me(data_E2[awmin:awmax], title='E2')
    # plot_me(data_T2[awmin:awmax], title='T2')

    # Clean up large memory
    del instru
    del data_X1
    del data_Y1
    del data_Z1
    del data_A1
    del data_E1
    del data_T1
    del data_X2
    del data_Y2
    del data_Z2
    del data_A2
    del data_E2
    del data_T2
    del tdi_raw
    del tdi_data

In [ ]:
for index in tqdm(range(first_instance,last_instance), desc=f'Processing {run_id}[{first_instance}:{last_instance}]', unit='instance'):
    t = Thread(target=process_instance, args=(index,))
    t.start()
    t.join()